# Cleaning users database

## Imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import Counter
import os
from dotenv import load_dotenv


## Loading database and discovery

In [6]:
# Load the users data
load_dotenv()
df_users = pd.read_csv(os.getenv("USER_DB_URL"), low_memory=False)

In [7]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207852 entries, 0 to 207851
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    207852 non-null  int64  
 1   locale                207852 non-null  object 
 2   name                  207842 non-null  object 
 3   statut_infolettre     207852 non-null  int64  
 4   statut_mailchimp      207666 non-null  object 
 5   public                207852 non-null  int64  
 6   prenom                12400 non-null   object 
 7   codepostal            135905 non-null  object 
 8   pays                  148899 non-null  object 
 9   anciennete            195169 non-null  float64
 10  statut                19061 non-null   object 
 11  fonction              214 non-null     object 
 12  fonction_longue       31 non-null      object 
 13  enseigne_en_eefe      207852 non-null  int64  
 14  aucun_etablissement   207852 non-null  int64  
 15  

In [8]:
df_users.shape

(207852, 24)

In [5]:
df_users.sample(3)

,id,locale,name,statut_infolettre,statut_mailchimp,public,prenom,codepostal,pays,anciennete,...,aucun_etablissement,json_niveau,json_discipline,json_etablissement,json_centre_interet,json_format,json_metadata,created_at,updated_at,date_derniere_action
130797,133659,fr,lauriane,1,subscribed,0,NaN,1280,France,12.0,...,0,"[""CP"", ""CE1"", ""CE2""]","[""Français"", ""Mathématiques"", ""Sciences et tec...","[{""id"": 53231, ""nom"": ""ECOLE ALTERNATIVE DU PA...",NaN,NaN,"[{""key"": ""newsletter"", ""value"": ""true"", ""creat...",2023-03-25 16:08:32,2023-10-28 10:27:32,2023-10-28 10:27:19
108867,111729,fr,Fadoua,0,subscribed,0,NaN,NaN,NaN,1.0,...,1,"[""CE1"", ""CE2"", ""CM1"", ""MS"", ""PS""]",[],[],NaN,NaN,"[{""key"": ""newsletter"", ""value"": ""false"", ""crea...",2022-08-25 19:59:03,2022-08-26 14:42:58,2022-08-26 16:42:54
122516,125378,fr,Liliana,0,subscribed,0,NaN,NaN,NaN,0.0,...,1,"[""CP"", ""CE1"", ""TPS"", ""CE2"", ""CM1"", ""PS"", ""MS"",...",[],[],NaN,NaN,"[{""key"": ""newsletter"", ""value"": ""false"", ""crea...",2022-12-09 06:15:29,2022-12-09 06:15:30,NaN


In [6]:
df_users.columns

Index(['id', 'locale', 'name', 'statut_infolettre', 'statut_mailchimp',
       'public', 'prenom', 'codepostal', 'pays', 'anciennete', 'statut',
       'fonction', 'fonction_longue', 'enseigne_en_eefe',
       'aucun_etablissement', 'json_niveau', 'json_discipline',
       'json_etablissement', 'json_centre_interet', 'json_format',
       'json_metadata', 'created_at', 'updated_at', 'date_derniere_action'],
      dtype='object')

## Let's drop unused columns

In [7]:
df_users.shape

(207852, 24)

### utils

In [8]:
def drop_col(string):
    if string in df_users.columns:
        df_users.drop(string, axis=1, inplace=True)
        return f"Column `{string}` dropped. New shape : {df_users.shape}"
    else:
        return f"Column `{string}` not found in DataFrame. Shape unchanged : {df_users.shape}"

### `locale`

`locale` is to determine if it's for Belgian people or other, because there is a special platform for Belgium.  
Let's drop Belgium

In [9]:
df_users = df_users[df_users.locale != "be"]
drop_col("locale")

'Column `locale` dropped. New shape : (204545, 23)'

### `public`

In [10]:
df_users.public.value_counts()

0    204545
Name: public, dtype: int64

It seems to be useless...

In [11]:
drop_col("public")

'Column `public` dropped. New shape : (204545, 22)'

### `name` and `prenom`

In [12]:
drop_col('name')
drop_col('prenom')

'Column `prenom` dropped. New shape : (204545, 20)'

### `pays`

In [13]:
df_users.pays.value_counts()

France                138273
Canada                   882
Maroc                    728
Suisse                   535
france                   438
                       ...  
France                     1
Dominican Republic         1
Malaysia                   1
fran                       1
Guinée-Équatoriale         1
Name: pays, Length: 507, dtype: int64

In [14]:
df_users.pays.isna().value_counts()

False    145592
True      58953
Name: pays, dtype: int64

oh my god... let's clean this !

In [15]:
df_users['pays'] = df_users['pays'].fillna('france')
df_users['pays'] = df_users['pays'].str.lower().str.strip()

In [16]:
pays = df_users.pays.unique()
pays.sort()
pays

array(['', '600', 'afghanistan', 'afrique du sud', 'aigueperse',
       'albanie', 'algeri', 'algeria', 'algerie', 'algérie',
       'algérie/ tlemcen', 'allemagne', 'amberieu en bugey', 'amiens',
       'andorre', 'angers', 'angleterre', 'angola', 'angouleme',
       'arabie saoudite', 'arabie saudite', 'argentina', 'argentine',
       'armenia', 'arménie', 'aubervilliers', 'australie', 'autriche',
       'azerbaïdjan', 'bahrain', 'bahreïn', 'banjul', 'begique', 'bel',
       'belgique', 'belgium', 'benin', 'bolivia', 'bolivie',
       'bosnia and herzegovi', 'bosnie-herzégovine', 'bouche', 'bourges',
       'brasil', 'brazil', 'bresil', 'bretagne', 'bretagne-france',
       'brlgique', 'brèsil', 'brésil', 'bulgarie', 'burkina faso',
       'burundi', 'bénin', 'caledonie', 'cambodge', 'cambodia',
       'cameroon', 'cameroun', 'canada', 'canada (québec)', 'cananda',
       'cap-vert', 'cape verde', 'cayenne', 'central african repu',
       'chaumont en vexin', 'chile', 'chili', 'china

In [17]:
len(pays)

370

In [ ]:
variants_france = [
    'france', 'f', 'fr', 'fra', 'fran', 'franc', 'francs', 'frnace',
    'frrance', 'fance', 'farnce', 'frane', 'francr', 'frande', 'franxe',
    'frannce', 'francec', 'francer', 'français', 'françe', 'francia',
    'francce', 'franccccccce', 'franche', 'france0', 'frankreich',
    "france  d'origine it", 'france (ile de la ré', 'france nouvelle-calé',
    'france réunion', 'france île de la réu', 'france/ gb/canada/us',
    'guadeloupe', 'guadeloupe (dom)', 'guadeloupe france',
    'martinique', 'martiniqie', 'martinique (france)', 'martinique ( france)',
    'réunion', 'reunion', 'runion', 'reunion france', 'la reunion', 'la réunion',
    'ile de la reunion', 'ile de la réunion', 'ile de la réunion (f',
    'île de la reunion',
    'guyane', 'guyane française', 'guyane francaise', 'french guiana',
    'mayotte', 'réside à mayotte',
    'nouvelle calédonie', 'nouvelle-calédonie', 'nouvelle calédonie',
    'nouvelle caledonie', 'nouvelle-caledonie', 'nouvelle camédonie',
    'nouvelle- calédonie', 'caledonie',
    'polynésie française', 'polynesie française', 'polynesie francaise',
    'polynésie', 'french polynesia', 'tahiti', 'tahiti (polynésie fr',
    'saint-martin', 'saint martin', 'saint-martin (partie française)',
    'saint barthelemy', 'saint pierre and miq',
    'maurice', 'ile maurice',
    'france (la réunion)', 'france (mayotte )', 'bretagne-france',
    'paris', 'nice', 'angers', 'strasbourg', 'bourges', 'tarbes',
    'montpellier', 'nantes', 'toulouse', 'mulhouse', 'cayenne', 'mirepoix',
    'eysines', 'aubervilliers', 'amiens', 'angouleme', 'vierzon',
    'le cannet', 'oyonnax', 'romans', 'chaumont en vexin', 'sucy',
    'amberieu en bugey', 'aigueperse', 'le creusot', 'mantrs la ville',
    'montbéliard', 'st joseph', 'bretagne', '', '600'
]

df_users.loc[df_users['pays'].isin(variants_france), 'pays'] = 'france'

df_users.loc[df_users['pays'].isin(['', '600']), 'pays'] = 'france'

In [19]:
pays = df_users.pays.unique()
pays.sort()
pays

array(['afghanistan', 'afrique du sud', 'albanie', 'algeri', 'algeria',
       'algerie', 'algérie', 'algérie/ tlemcen', 'allemagne', 'andorre',
       'angleterre', 'angola', 'arabie saoudite', 'arabie saudite',
       'argentina', 'argentine', 'armenia', 'arménie', 'australie',
       'autriche', 'azerbaïdjan', 'bahrain', 'bahreïn', 'banjul',
       'begique', 'bel', 'belgique', 'belgium', 'benin', 'bolivia',
       'bolivie', 'bosnia and herzegovi', 'bosnie-herzégovine', 'bouche',
       'brasil', 'brazil', 'bresil', 'brlgique', 'brèsil', 'brésil',
       'bulgarie', 'burkina faso', 'burundi', 'bénin', 'cambodge',
       'cambodia', 'cameroon', 'cameroun', 'canada', 'canada (québec)',
       'cananda', 'cap-vert', 'cape verde', 'central african repu',
       'chile', 'chili', 'china', 'chine', 'chypre', 'colombia',
       'colombie', 'comores', 'congo', 'congo (rép. dém.)', 'corée',
       'corée du sud', 'costa rica', 'cote d ivoire', "cote d' ivoire",
       "cote d'ivoire", 'croa

In [20]:
df_users.pays.value_counts()

france                  198895
canada                     904
maroc                      835
suisse                     547
belgium                    257
                         ...  
rdcongo                      1
pays                         1
iran, islamic republ         1
cape verde                   1
guinée-équatoriale           1
Name: pays, Length: 261, dtype: int64

In [21]:
df_users = df_users[df_users.pays == 'france']
df_users.shape

(198895, 20)

In [22]:
df_users.pays.value_counts()

france    198895
Name: pays, dtype: int64

In [23]:
drop_col('pays')

'Column `pays` dropped. New shape : (198895, 19)'

### `statut`

In [24]:
df_users.statut.value_counts()

Contractuel                                 6350
Fonctionnaire stagiaire                     4806
Titulaire                                   2761
Préparationnaire au concours                2211
Etudiant en M1 MEEF                         1179
Etudiant en M2 MEEF                          891
Etudiant Contractuel Alternant (M2 MEEF)     761
Name: statut, dtype: int64

Good to know : users never updates there profile... So... it can't be used. Let's drop this column

In [25]:
drop_col('statut')

'Column `statut` dropped. New shape : (198895, 18)'

### `fonction`

In [26]:
df_users.fonction.value_counts()

Professeure des écoles en maternelle                                                                                       2
Formatrice chez ENQUÊTE                                                                                                    2
Professeure des écoles depuis 2012                                                                                         2
Professeur en lycée professionnel depuis 2012                                                                              2
Professeure documentaliste en collège depuis 1997 et facilitatrice qualifiée en discipline positive                        2
                                                                                                                          ..
Professeure de mathématiques depuis 2005 et chargée de mission pour la CARDIE                                              1
Coordonnatrice Ulis au collège depuis 2020                                                                                 1


In [27]:
df_users.fonction.isna().value_counts()

True     198696
False       199
Name: fonction, dtype: int64

In [28]:
df_users.fonction_longue.value_counts()

Professeure des écoles en élémentaire depuis 2019                                                                                                                                                                                                                                                                                                                                                              2
Jocelyne RAJOHNSON DELAN \n- ⁠Professeure des écoles depuis 2019. \n- ⁠Passionée par le monde de l’éducation et de la parentalité, je suis également Psychopédagogue et Consultante en Parentalité Positive pour accompagner parents et enfants. \n- ⁠Mon compte Instagram: 🐨 Monpetitnideveil 🐨\n                                                                                                             1
Professeur des écoles en Bretagne depuis 2009, j’ai enseigné en cycle 3, puis en cycle 2, avant de revenir au cycle 3. Actuellement en CM1, je mets en œuvre une pédagogie personnalisée et un enseign

In [29]:
df_users.fonction_longue.isna().value_counts()

True     198876
False        19
Name: fonction_longue, dtype: int64

It's useless, it's just for some author on website → Let's drop !

In [30]:
drop_col('fonction')

'Column `fonction` dropped. New shape : (198895, 17)'

In [31]:
drop_col('fonction_longue')

'Column `fonction_longue` dropped. New shape : (198895, 16)'

### `enseigne_en_eefe`

In [32]:
df_users.enseigne_en_eefe.value_counts()

0    198886
1         9
Name: enseigne_en_eefe, dtype: int64

Not a lot of people → Let's drop

In [33]:
drop_col('enseigne_en_eefe')

'Column `enseigne_en_eefe` dropped. New shape : (198895, 15)'

### `date_derniere_action` 

It will be determined with the `interactions` table. So let's drop.  
Same for `updated_at` 

In [34]:
drop_col('date_derniere_action')

'Column `date_derniere_action` dropped. New shape : (198895, 14)'

In [35]:
drop_col('updated_at')

'Column `updated_at` dropped. New shape : (198895, 13)'

### `json_format`

In [36]:
df_users.json_format.value_counts()

[]                             31657
["PDF"]                           78
["Vidéo", "Article", "PDF"]       62
["Article", "PDF"]                47
["PDF", "Article"]                35
["PDF", "Article", "Vidéo"]       27
["Vidéo", "PDF"]                  26
["Vidéo"]                         25
["PDF", "Vidéo"]                  21
["Article"]                       13
["Vidéo", "PDF", "Article"]        8
["PDF", "Vidéo", "Article"]        7
["Article", "PDF", "Vidéo"]        6
["Vidéo", "Article"]               6
["Article", "Vidéo"]               4
["Article", "Vidéo", "PDF"]        2
Name: json_format, dtype: int64

It seems to be an old column, with few answers. → Let's drop

In [37]:
drop_col('json_format')

'Column `json_format` dropped. New shape : (198895, 12)'

### `json_centre_interet`

In [38]:
df_users.json_centre_interet.value_counts()

[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               85972
["Organisation", "Politiques éducatives", "Théorie des apprentissages", "Neurosciences", "Psychologie de l’enfant", "Didactique", "Numérique", "C

In [39]:
df_users.json_centre_interet.isna().value_counts()

False    101055
True      97840
Name: json_centre_interet, dtype: int64

Around 100 000 with no info. And 85 000 with all selected. → Let's drop !

In [40]:
drop_col('json_centre_interet')

'Column `json_centre_interet` dropped. New shape : (198895, 11)'

## Explore other columns

### `json_niveau`

In [41]:
df_users.json_niveau.unique()

array(['["MS"]', '[]', '["PS"]', ..., '["CM1","CM2","CP"]',
       '["2nde","5e","1ère","3e","Terminale"]', '["CP","PS","CE1"]'],
      dtype=object)

In [42]:
df_users.json_niveau.isna().value_counts()

False    198891
True          4
Name: json_niveau, dtype: int64

#### Clean Niveau

In [43]:
explo_niveaux = df_users.json_niveau.dropna().unique()

In [44]:
explo_niveaux.sort()
explo_niveaux

array(['["", "PS"]', '["1ère", "2nde", "4e", "3e"]',
       '["1ère", "2nde", "POST BAC"]', ..., '["Études supérieures"]',
       '["Étudiant stagiaire"]', '[]'], dtype=object)

In [45]:
(df_users['json_niveau'] == '[]').sum()

6692

In [46]:
all_niveaux = set()

for niveaux_str in df_users['json_niveau'].dropna():
    try:
        niveaux_list = json.loads(niveaux_str)
        all_niveaux.update(niveaux_list)  # Ajouter tous les éléments au set
    except:
        print(f"Erreur de parsing pour : {niveaux_str}")

In [47]:
display(all_niveaux)
display(len(all_niveaux))

{'',
 '1ère',
 '2nde',
 '3e',
 '4e',
 '5e',
 '6e',
 '6ème primaire',
 'ASH',
 'Autres',
 'Bac Pro',
 'CAP',
 'CE1',
 'CE2',
 'CM1',
 'CM2',
 'CP',
 'Collège',
 'Direction',
 'Elémentaire',
 'Enseignement spécialisé',
 'Formateur-trice /Inspecteur-trice',
 'GS',
 'Lycée',
 'MS',
 'Maternelle',
 'POST BAC',
 'PS',
 'Professeur-e documentaliste',
 'SEGPA',
 'TPS',
 'Terminale',
 'Études supérieures',
 'Étudiant stagiaire'}

34

In [48]:
# Compter chaque niveau individuellement
niveau_counts = Counter()

for niveaux_str in df_users['json_niveau'].dropna():
    niveaux_list = json.loads(niveaux_str)
    niveau_counts.update(niveaux_list)

print("Fréquence de chaque niveau :")
for niveau, count in niveau_counts.most_common():
    print(f"  {niveau}: {count}")

Fréquence de chaque niveau :
  3e: 26328
  6e: 26078
  4e: 25041
  5e: 24667
  GS: 23100
  CE2: 22502
  MS: 22077
  CM1: 21916
  CE1: 21838
  CP: 21649
  CM2: 20864
  PS: 19899
  2nde: 14123
  Bac Pro: 12398
  1ère: 12099
  Terminale: 11928
  ASH: 8133
  TPS: 6931
  POST BAC: 5478
  CAP: 5331
  Formateur-trice /Inspecteur-trice: 5250
  SEGPA: 2586
  Direction: 1873
  Professeur-e documentaliste: 942
  Études supérieures: 7
  Enseignement spécialisé: 7
  Maternelle: 4
  Elémentaire: 2
  Collège: 2
  Lycée: 2
  Étudiant stagiaire: 2
  6ème primaire: 2
  : 1
  Autres: 1


In [ ]:
categories_rares = [
    "Études supérieures", "Enseignement spécialisé", "Maternelle",
    "Elémentaire", "Collège", "Lycée", "Étudiant stagiaire",
    "6ème primaire", "", "Autres"
]

def contains_rare_category(niveaux_str):
    if pd.isna(niveaux_str):
        return False
    niveaux_list = json.loads(niveaux_str)
    return any(niveau in categories_rares for niveau in niveaux_list)

mask_rare = df_users['json_niveau'].apply(contains_rare_category)

df_rare = df_users[mask_rare].copy()
df_rare

,id,statut_infolettre,statut_mailchimp,codepostal,anciennete,aucun_etablissement,json_niveau,json_discipline,json_etablissement,json_metadata,created_at
6,7,1,subscribed,31130,25.0,0,"[""CP"", ""Elémentaire"", ""CE1"", ""CE2"", ""CM1"", ""PS...",[],"[{""id"": 1901, ""nom"": ""Ecole élémentaire publiq...","[{""key"":""newsletter"",""value"":""undefined"",""crea...",2017-01-15 16:50:11
780,781,1,subscribed,94400,3.0,1,"["""", ""PS""]",[],NaN,"[{""key"": ""parcoursOffertCanvasId"", ""value"": 44...",2017-08-17 09:16:35
20180,20386,1,unsubscribed,NaN,0.0,1,"[""TPS"", ""PS"", ""MS"", ""GS"", ""Maternelle""]",[],NaN,"[{""key"": ""programme_thematique"", ""value"": null...",2019-04-24 08:22:53
40120,42074,0,subscribed,31060,18.0,0,"[""Études supérieures""]",[],"[{""id"": 1968, ""nom"": ""Lycée professionnel priv...","[{""key"": ""inscrit_caprentree"", ""value"": ""true""...",2020-08-20 12:00:56
85286,88019,1,subscribed,72405,15.0,1,"[""TPS"", ""PS"", ""MS"", ""GS"", ""Maternelle"", ""CP"", ...","[""Mathématiques"", ""Sciences et technologies""]",[],"[{""key"": ""etablissement_1"", ""value"": """", ""crea...",2021-10-21 13:06:33
90263,93125,1,subscribed,NaN,0.0,1,"[""TPS"",""PS"",""MS"",""GS"",""Maternelle"",""Direction""]",[],[],"[{""key"":""newsletter"",""value"":""true"",""created_a...",2022-02-08 09:04:53
114254,117116,1,subscribed,NaN,NaN,1,"[""6e"", ""5e"", ""4e"", ""3e"", ""Collège""]",[],[],"[{""key"": ""neo2023"", ""value"": ""true"", ""created_...",2022-09-29 19:10:51
119129,121991,0,subscribed,82500,20.0,0,"[""ASH"", ""Enseignement spécialisé""]",[],"[{""id"": 37170, ""nom"": ""Collège Théodore Despey...","[{""key"": ""newsletter"", ""value"": ""undefined"", ""...",2022-11-04 08:48:18
140885,143747,0,subscribed,95680,1.0,0,"[""Enseignement spécialisé""]",[],"[{""id"": 19224, ""nom"": ""Ecole primaire publique...","[{""key"": ""newsletter"", ""value"": ""false"", ""crea...",2023-07-19 15:05:16
142145,145007,1,subscribed,51450,3.0,0,"[""2nde"", ""1ère"", ""Terminale"", ""Bac Pro"", ""Lycée""]","[""Histoire et géographie""]","[{""id"": 7659, ""nom"": ""Ecole maternelle la riba...","[{""key"": ""newsletter"", ""value"": ""undefined"", ""...",2023-07-25 10:07:34


In [ ]:
categories_rares = [
    "Études supérieures", "Enseignement spécialisé", "Maternelle",
    "Elémentaire", "Collège", "Lycée", "Étudiant stagiaire",
    "6ème primaire", "", "Autres"
]

def clean_json_niveau(niveaux_str):
    if pd.isna(niveaux_str):
        return niveaux_str

    try:
        niveaux_list = json.loads(niveaux_str)

        # Séparer les éléments rares des éléments valides
        elements_rares = [niveau for niveau in niveaux_list if niveau in categories_rares]
        elements_valides = [niveau for niveau in niveaux_list if niveau not in categories_rares]

        # Si il y a des éléments valides, on garde que ceux-là
        if elements_valides:
            return json.dumps(elements_valides, ensure_ascii=False)
        # Sinon, on garde les éléments rares (cas où il n'y a que ça)
        else:
            return json.dumps(elements_rares, ensure_ascii=False)

    except:
        return niveaux_str

# Appliquer le nettoyage
df_users['json_niveau'] = df_users['json_niveau'].apply(clean_json_niveau)

In [51]:
mask_rare = df_users['json_niveau'].apply(contains_rare_category)

df_rare = df_users[mask_rare].copy()
df_rare

,id,statut_infolettre,statut_mailchimp,codepostal,anciennete,aucun_etablissement,json_niveau,json_discipline,json_etablissement,json_metadata,created_at
40120,42074,0,subscribed,31060,18.0,0,"[""Études supérieures""]",[],"[{""id"": 1968, ""nom"": ""Lycée professionnel priv...","[{""key"": ""inscrit_caprentree"", ""value"": ""true""...",2020-08-20 12:00:56
140885,143747,0,subscribed,95680,1.0,0,"[""Enseignement spécialisé""]",[],"[{""id"": 19224, ""nom"": ""Ecole primaire publique...","[{""key"": ""newsletter"", ""value"": ""false"", ""crea...",2023-07-19 15:05:16
196802,199671,1,subscribed,NaN,42.0,1,"[""Enseignement spécialisé""]","[""Arts""]",[],"[{""key"": ""newsletter"", ""value"": ""true"", ""creat...",2024-12-20 07:55:41
196805,199674,0,subscribed,NaN,19.0,1,"[""Études supérieures""]","[""Mathématiques""]",[],"[{""key"": ""newsletter"", ""value"": ""undefined"", ""...",2024-12-20 10:51:31
196813,199682,0,subscribed,NaN,25.0,1,"[""Études supérieures""]",[],[],"[{""key"": ""newsletter"", ""value"": ""false"", ""crea...",2024-12-20 18:41:48
196856,199725,0,unsubscribed,NaN,1.0,1,"[""Études supérieures""]",[],[],"[{""key"": ""newsletter"", ""value"": false, ""create...",2024-12-22 22:02:54
196874,199743,0,subscribed,NaN,59.0,1,"[""Études supérieures""]",[],[],"[{""key"": ""newsletter"", ""value"": ""false"", ""crea...",2024-12-23 16:00:18
196885,199754,1,subscribed,NaN,32.0,1,"[""Enseignement spécialisé""]","[""Mathématiques""]",[],"[{""key"": ""newsletter"", ""value"": ""undefined"", ""...",2024-12-24 10:11:40
196922,199791,1,subscribed,NaN,0.0,1,"[""Études supérieures""]",[],[],"[{""key"": ""newsletter"", ""value"": ""true"", ""creat...",2024-12-26 08:26:42
196942,199811,0,subscribed,91170,27.0,0,"[""Enseignement spécialisé""]",[],"[{""id"": 11854, ""nom"": ""Ecole élémentaire Rolan...","[{""key"": ""newsletter"", ""value"": ""false"", ""crea...",2024-12-27 09:30:35


In [ ]:
niveaux_rares_remain = set()

for niveaux_str in df_rare['json_niveau'].dropna():
    try:
        niveaux_list = json.loads(niveaux_str)
        niveaux_rares_remain.update(niveaux_list)
    except:
        print(f"Erreur de parsing pour : {niveaux_str}")

In [53]:
niveaux_rares_remain

{'6ème primaire',
 'Enseignement spécialisé',
 'Études supérieures',
 'Étudiant stagiaire'}

In [ ]:
def replace_categories(niveaux_str):
    if pd.isna(niveaux_str):
        return niveaux_str

    try:
        niveaux_list = json.loads(niveaux_str)

        # Remplacer les catégories
        niveaux_replaced = []
        for niveau in niveaux_list:
            if niveau == "Enseignement spécialisé":
                niveaux_replaced.append("ASH")
            elif niveau == "Études supérieures":
                niveaux_replaced.append("POST BAC")
            else:
                niveaux_replaced.append(niveau)

        return json.dumps(niveaux_replaced, ensure_ascii=False)

    except:
        return niveaux_str

df_users['json_niveau'] = df_users['json_niveau'].apply(replace_categories)


In [55]:
mask_rare = df_users['json_niveau'].apply(contains_rare_category)

df_rare = df_users[mask_rare].copy()
df_rare

,id,statut_infolettre,statut_mailchimp,codepostal,anciennete,aucun_etablissement,json_niveau,json_discipline,json_etablissement,json_metadata,created_at
196948,199817,0,subscribed,92250,1.0,0,"[""6ème primaire""]",[],"[{""id"": 11306, ""nom"": ""Ecole élémentaire publi...","[{""key"": ""newsletter"", ""value"": ""undefined"", ""...",2024-12-27 13:42:09
197022,199891,1,subscribed,NaN,0.0,1,"[""Étudiant stagiaire""]","[""Enseignement professionnel""]",[],"[{""key"": ""newsletter"", ""value"": ""true"", ""creat...",2024-12-30 19:12:44


In [56]:
df_users = df_users.drop([196948, 197022])

In [57]:
df_users = df_users.reset_index(drop=True)

In [58]:
all_niveaux = set()

for niveaux_str in df_users['json_niveau'].dropna():
    try:
        niveaux_list = json.loads(niveaux_str)
        all_niveaux.update(niveaux_list)  # Ajouter tous les éléments au set
    except:
        print(f"Erreur de parsing pour : {niveaux_str}")
display(all_niveaux)
display(len(all_niveaux))

{'1ère',
 '2nde',
 '3e',
 '4e',
 '5e',
 '6e',
 'ASH',
 'Bac Pro',
 'CAP',
 'CE1',
 'CE2',
 'CM1',
 'CM2',
 'CP',
 'Direction',
 'Formateur-trice /Inspecteur-trice',
 'GS',
 'MS',
 'POST BAC',
 'PS',
 'Professeur-e documentaliste',
 'SEGPA',
 'TPS',
 'Terminale'}

24

In [59]:
# Compter chaque niveau individuellement
niveau_counts = Counter()

for niveaux_str in df_users['json_niveau'].dropna():
    niveaux_list = json.loads(niveaux_str)
    niveau_counts.update(niveaux_list)

print("Fréquence de chaque niveau :")
for niveau, count in niveau_counts.most_common():
    print(f"  {niveau}: {count}")

Fréquence de chaque niveau :
  3e: 26328
  6e: 26078
  4e: 25041
  5e: 24667
  GS: 23100
  CE2: 22502
  MS: 22077
  CM1: 21916
  CE1: 21838
  CP: 21649
  CM2: 20864
  PS: 19899
  2nde: 14123
  Bac Pro: 12398
  1ère: 12099
  Terminale: 11928
  ASH: 8139
  TPS: 6931
  POST BAC: 5485
  CAP: 5331
  Formateur-trice /Inspecteur-trice: 5250
  SEGPA: 2586
  Direction: 1873
  Professeur-e documentaliste: 942


#### OneHotEncode Niveaux

In [60]:
all_niveaux

{'1ère',
 '2nde',
 '3e',
 '4e',
 '5e',
 '6e',
 'ASH',
 'Bac Pro',
 'CAP',
 'CE1',
 'CE2',
 'CM1',
 'CM2',
 'CP',
 'Direction',
 'Formateur-trice /Inspecteur-trice',
 'GS',
 'MS',
 'POST BAC',
 'PS',
 'Professeur-e documentaliste',
 'SEGPA',
 'TPS',
 'Terminale'}

In [61]:
# Drop where niveaux is NaN
df_users = df_users.drop([33664, 33721, 33745, 52368])
df_users = df_users.reset_index(drop=True)

In [62]:
for niveau in sorted(all_niveaux):
    col_name = f"niveau_{niveau}".replace(" ", "_").replace("-", "_").replace("è", "e").replace("é", "e").lower()
    df_users[col_name] = 0

In [63]:
for idx, niveaux_str in df_users['json_niveau'].items():
    niveaux_list = json.loads(niveaux_str)
    for niveau in niveaux_list:
        col_name = f"niveau_{niveau}".replace(" ", "_").replace("-", "_").replace("è", "e").replace("é", "e").lower()
        if col_name in df_users.columns:
            df_users.loc[idx, col_name] = 1

####  Encode `degre`

In [ ]:
# Définir les niveaux par degré
niveaux_primaires = ['TPS', 'PS', 'MS', 'GS', 'CP', 'CE1', 'CE2', 'CM1', 'CM2', 'Direction', 'ASH']
niveaux_secondaires = ['6e', '5e', '4e', '3e', '2nde', '1ère', 'Terminale', 'Bac Pro', 'CAP', 'SEGPA', 'Professeur-e documentaliste', 'POST BAC']
niveaux_formateurs = ['Formateur-trice /Inspecteur-trice']

# Créer la colonne degre (0 par défaut)
df_users['degre'] = 0

# Remplir la colonne
for idx, niveaux_str in df_users['json_niveau'].items():
    if isinstance(niveaux_str, str):
        try:
            niveaux_list = json.loads(niveaux_str)

            # Compter par catégorie
            count_primaire = sum(1 for niveau in niveaux_list if niveau in niveaux_primaires)
            count_secondaire = sum(1 for niveau in niveaux_list if niveau in niveaux_secondaires)
            count_formateur = sum(1 for niveau in niveaux_list if niveau in niveaux_formateurs)

            # Logique de priorité : formateur > majorité primaire/secondaire
            if count_formateur > 0:
                df_users.loc[idx, 'degre'] = 3  # Formateur
            elif count_primaire > count_secondaire:
                df_users.loc[idx, 'degre'] = 1  # Primaire
            elif count_secondaire > count_primaire:
                df_users.loc[idx, 'degre'] = 2  # Secondaire
            elif count_primaire == count_secondaire and count_primaire > 0:
                df_users.loc[idx, 'degre'] = 1  # En cas d'égalité, primaire par défaut

        except:
            pass

#### Encode `grandsNiveaux`

In [ ]:
niveaux_maternelle = ['TPS', 'PS', 'MS', 'GS', 'Direction','ASH']
niveaux_elementaire = ['CP', 'CE1', 'CE2', 'CM1', 'CM2', 'ASH', 'Direction']
niveaux_college = ['6e', '5e', '4e', '3e', 'SEGPA', 'Professeur-e documentaliste']
niveaux_lycee = ['2nde', '1ère', 'Terminale', 'Professeur-e documentaliste']
niveaux_lycee_pro = ['Bac Pro', 'CAP']
niveaux_autre = ['POST BAC', 'Formateur-trice /Inspecteur-trice']

etablissements = ['maternelle', 'elementaire', 'college', 'lycee', 'lycee_pro', 'autre']

for etab in etablissements:
    df_users[etab] = 0

for idx, niveaux_str in df_users['json_niveau'].items():
    if isinstance(niveaux_str, str):
        try:
            niveaux_list = json.loads(niveaux_str)

            if any(niveau in niveaux_maternelle for niveau in niveaux_list):
                df_users.loc[idx, 'maternelle'] = 1
            if any(niveau in niveaux_elementaire for niveau in niveaux_list):
                df_users.loc[idx, 'elementaire'] = 1
            if any(niveau in niveaux_college for niveau in niveaux_list):
                df_users.loc[idx, 'college'] = 1
            if any(niveau in niveaux_lycee for niveau in niveaux_list):
                df_users.loc[idx, 'lycee'] = 1
            if any(niveau in niveaux_lycee_pro for niveau in niveaux_list):
                df_users.loc[idx, 'lycee_pro'] = 1
            if any(niveau in niveaux_autre for niveau in niveaux_list):
                df_users.loc[idx, 'autre'] = 1

        except:
            pass

print("Distribution par établissement :")
for etab in etablissements:
    count = df_users[etab].sum()
    print(f"{etab}: {count}")

Distribution par établissement :
maternelle: 52119
elementaire: 77946
college: 46890
lycee: 21604
lycee_pro: 14763
autre: 10539


#### Drop `json_niveau`

In [74]:
df_users = df_users.drop('json_niveau', axis=1)

### `json_etablissement`

In [79]:
# Extract 10 to see how it's done

for i, (idx, etab) in enumerate(df_users['json_etablissement'].dropna().sample(10, random_state=42).items(), 1):
    print(f"\n{i}. Index {idx}:")
    print(f"   {etab}")

# Voir quelques stats générales
print(f"\n" + "=" * 50)
print(f"Total d'entrées json_etablissement non-nulles : {df_users['json_etablissement'].notna().sum()}")
print(f"Valeurs uniques : {df_users['json_etablissement'].nunique()}")


1. Index 182910:
   []

2. Index 174124:
   [{"id": 46564, "nom": "Ecole primaire privée Montessori de Lyon", "eefe": false, "pays": "France", "ville": "Lyon 6e  Arrondissement", "region": "Auvergne-Rhône-Alpes", "adresse": "8 rue Barrier", "academie": "Lyon", "code_pays": "FRA", "code_postal": "69006", "code_etablissement": "0693085D", "etat_etablissement": "OUVERT", "type_etablissement": "ECOLE DE NIVEAU ELEMENTAIRE", "code_type_etablissement": "151"}]

3. Index 180018:
   []

4. Index 85169:
   [{"id": 48358, "nom": "Collège Jules Verne", "ville": "Les Mureaux", "region": "Ile-de-France", "adresse": "Rue Albert Thomas", "academie": "Versailles", "code_postal": "78130", "code_etablissement": "0780180X", "etat_etablissement": "OUVERT", "type_etablissement": "COLLEGE"}]

5. Index 146421:
   [{"id": 17424, "nom": "Ecole Primaire", "eefe": false, "pays": "France", "ville": "Périssac", "region": "Nouvelle-Aquitaine", "adresse": "9 avenue des Ecoles", "academie": "Bordeaux", "code_pays": 

In [ ]:
def extract_etablissement_info(etab_str):
    if pd.isna(etab_str)or etab_str == '[]':
        return np.nan, np.nan, np.nan

    try:
        etab_list = json.loads(etab_str)

        # Vérifier qu'on a bien une liste non vide
        if isinstance(etab_list, list) and len(etab_list) > 0:
            etab_obj = etab_list[0]  # Prendre le premier (et probablement unique) établissement

            # Extraire les infos avec valeur par défaut 'NR'
            code_postal = etab_obj.get('code_postal', 'NR')
            academie = etab_obj.get('academie', 'NR')
            type_etablissement = etab_obj.get('type_etablissement', 'NR')

            return code_postal, academie, type_etablissement
        else:
            return 'NR', 'NR', 'NR'

    except:
        return 'NR', 'NR', 'NR'

# Appliquer l'extraction et créer les 3 colonnes
df_users[['code_postal_etab', 'academie_etab', 'type_etablissement_etab']] = df_users['json_etablissement'].apply(
    lambda x: pd.Series(extract_etablissement_info(x))
)

In [81]:
df_users[['code_postal_etab', 'academie_etab', 'type_etablissement_etab']]

,code_postal_etab,academie_etab,type_etablissement_etab
0,78770,Versailles,ECOLE MATERNELLE
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,93260,Créteil,ECOLE MATERNELLE
4,NaN,NaN,NaN
...,...,...,...
198884,69270,Lyon,ECOLE DE NIVEAU ELEMENTAIRE
198885,NaN,NaN,NaN
198886,49270,Nantes,ECOLE DE NIVEAU ELEMENTAIRE
198887,52210,Reims,ECOLE DE NIVEAU ELEMENTAIRE


In [ ]:
def complete_codepostal(row):
    codepostal = row['codepostal']
    code_postal_etab = row['code_postal_etab']

    # Cas 1: codepostal NaN et code_postal_etab NaN → NaN
    if pd.isna(codepostal) and pd.isna(code_postal_etab):
        return np.nan

    # Cas 2: codepostal NaN et code_postal_etab a une valeur → code_postal_etab
    elif pd.isna(codepostal) and pd.notna(code_postal_etab):
        return code_postal_etab

    # Cas 3: codepostal a une valeur et code_postal_etab NaN → ne rien faire (garder codepostal)
    elif pd.notna(codepostal) and pd.isna(code_postal_etab):
        return codepostal

    # Cas 4: codepostal a une valeur et code_postal_etab a une valeur → code_postal_etab (priorité)
    elif pd.notna(codepostal) and pd.notna(code_postal_etab):
        return code_postal_etab

df_users['codepostal'] = df_users.apply(complete_codepostal, axis=1)

In [111]:
df_users.codepostal.isna().value_counts()

False    132194
True      66695
Name: codepostal, dtype: int64

In [112]:
df_users.code_postal_etab.isna().value_counts()

False    101049
True      97840
Name: code_postal_etab, dtype: int64

**Drop `aucun_etablissement`**

In [113]:
drop_col('aucun_etablissement')

'Column `aucun_etablissement` dropped. New shape : (198889, 43)'

**Drop `json_etablissement`**

In [120]:
drop_col('json_etablissement')

'Column `json_etablissement` dropped. New shape : (198889, 42)'

#### Create `departement`

In [ ]:
def extract_departement(code_postal):
    if pd.isna(code_postal):
        return np.nan

    # Convertir en string et s'assurer qu'on a 5 chiffres (zfill pour ajouter les 0)
    cp = str(code_postal).strip().zfill(5)

    # Vérifier que c'est numérique après padding
    if not cp.isdigit():
        return np.nan

    # Cas spéciaux des DOM-TOM (3 chiffres)
    if cp.startswith('97') or cp.startswith('98'):
        return cp[:3]  # 971, 972, 973, 974, 975, 976, 977, 978, 984, 986, 987, 988

    # Cas général (2 chiffres) - garder le zéro initial
    return cp[:2]

df_users['departement'] = df_users['codepostal'].apply(extract_departement)

#### complete `academie_etab`

In [ ]:
dept_to_academie = {
    # Clermont-Ferrand
    '03': 'Clermont-Ferrand', '15': 'Clermont-Ferrand', '43': 'Clermont-Ferrand', '63': 'Clermont-Ferrand',

    # Grenoble
    '07': 'Grenoble', '26': 'Grenoble', '38': 'Grenoble', '73': 'Grenoble', '74': 'Grenoble',

    # Lyon
    '01': 'Lyon', '42': 'Lyon', '69': 'Lyon', '69D': 'Lyon', '69M': 'Lyon',

    # Besançon
    '25': 'Besançon', '39': 'Besançon', '70': 'Besançon', '90': 'Besançon',

    # Dijon
    '21': 'Dijon', '58': 'Dijon', '71': 'Dijon', '89': 'Dijon',

    # Rennes
    '22': 'Rennes', '29': 'Rennes', '35': 'Rennes', '56': 'Rennes',

    # Orléans-Tours
    '18': 'Orléans-Tours', '28': 'Orléans-Tours', '36': 'Orléans-Tours', '37': 'Orléans-Tours', '41': 'Orléans-Tours', '45': 'Orléans-Tours',

    # Corse
    '2A': 'Corse', '2B': 'Corse',

    # Nancy-Metz
    '54': 'Nancy-Metz', '55': 'Nancy-Metz', '57': 'Nancy-Metz', '88': 'Nancy-Metz',

    # Reims
    '08': 'Reims', '10': 'Reims', '51': 'Reims', '52': 'Reims',

    # Strasbourg
    '67': 'Strasbourg', '68': 'Strasbourg',

    # Guadeloupe
    '971': 'Guadeloupe', '977': 'Guadeloupe', '978': 'Guadeloupe',

    # Guyane
    '973': 'Guyane',

    # Amiens
    '02': 'Amiens', '60': 'Amiens', '80': 'Amiens',

    # Lille
    '59': 'Lille', '62': 'Lille',

    # Créteil
    '77': 'Créteil', '93': 'Créteil', '94': 'Créteil',

    # Paris
    '75': 'Paris',

    # Versailles
    '78': 'Versailles', '91': 'Versailles', '92': 'Versailles', '95': 'Versailles',

    # Martinique
    '972': 'Martinique',

    # Normandie
    '14': 'Normandie', '27': 'Normandie', '50': 'Normandie', '61': 'Normandie', '76': 'Normandie', '975': 'Normandie',

    # Bordeaux
    '24': 'Bordeaux', '33': 'Bordeaux', '40': 'Bordeaux', '47': 'Bordeaux', '64': 'Bordeaux',

    # Limoges
    '19': 'Limoges', '23': 'Limoges', '87': 'Limoges',

    # Poitiers
    '16': 'Poitiers', '17': 'Poitiers', '79': 'Poitiers', '86': 'Poitiers',

    # Montpellier
    '11': 'Montpellier', '30': 'Montpellier', '34': 'Montpellier', '48': 'Montpellier', '66': 'Montpellier',

    # Toulouse
    '09': 'Toulouse', '12': 'Toulouse', '31': 'Toulouse', '32': 'Toulouse', '46': 'Toulouse', '65': 'Toulouse', '81': 'Toulouse', '82': 'Toulouse',

    # Nantes
    '44': 'Nantes', '49': 'Nantes', '53': 'Nantes', '72': 'Nantes', '85': 'Nantes',

    # Aix-Marseille
    '04': 'Aix-Marseille', '05': 'Aix-Marseille', '13': 'Aix-Marseille', '84': 'Aix-Marseille',

    # Nice
    '06': 'Nice', '83': 'Nice',

    # La Réunion
    '974': 'La Réunion',

    # Mayotte
    '976': 'Mayotte'
}

def complete_academie(row):
    academie_etab = row['academie_etab']
    departement = row['departement']

    if pd.notna(academie_etab):
        return academie_etab

    if pd.notna(departement) and departement in dept_to_academie:
        return dept_to_academie[departement]

    return np.nan

df_users['academie_etab'] = df_users.apply(complete_academie, axis=1)

### `json_discipline`

In [ ]:
def extract_discipline(row):
    maternelle = row['maternelle']
    elementaire = row['elementaire']
    json_discipline = row['json_discipline']

    # Si maternelle ou élémentaire = 1, on met NaN
    if maternelle == 1 or elementaire == 1:
        return np.nan

    # Sinon, on extrait la première discipline du JSON
    if pd.notna(json_discipline):
        try:
            discipline_list = json.loads(json_discipline)

            # Vérifier qu'on a bien une liste non vide
            if isinstance(discipline_list, list) and len(discipline_list) > 0:
                return discipline_list[0]  # Premier élément au format string
            else:
                return np.nan
        except:
            return np.nan

    return np.nan

# Créer la colonne discipline
df_users['discipline'] = df_users.apply(extract_discipline, axis=1)

In [126]:
drop_col('json_discipline')

'Column `json_discipline` dropped. New shape : (198889, 43)'

### Explore `json_metadata`

In [130]:
# Extract 10 to see how it's done

for i, (idx, etab) in enumerate(df_users['json_metadata'].dropna().sample(10, random_state=42).items(), 1):
    print(f"\n{i}. Index {idx}:")
    print(f"   {etab}")

# Voir quelques stats générales
print(f"\n" + "=" * 50)
print(f"Total d'entrées json_metadata non-nulles : {df_users['json_metadata'].notna().sum()}")
print(f"Valeurs uniques : {df_users['json_metadata'].nunique()}")


1. Index 124196:
   [{"key": "newsletter", "value": "false", "created_at": "2023-02-03T22:01:53.360Z", "updated_at": "2023-02-03T22:03:27.960Z"}, {"key": "etablissement_1", "value": "0100375D - Ecole élémentaire Saint-Exupéry 10420 - Academie de Reims", "created_at": "2023-02-03T22:01:53.360Z", "updated_at": "2023-02-03T22:03:27.960Z"}, {"key": "etablissement_2", "value": "", "created_at": "2023-02-03T22:01:53.360Z", "updated_at": "2023-02-03T22:03:27.960Z"}, {"key": "etablissement_3", "value": "", "created_at": "2023-02-03T22:01:53.360Z", "updated_at": "2023-02-03T22:03:27.960Z"}, {"key": "etablissement_4", "value": "", "created_at": "2023-02-03T22:01:53.360Z", "updated_at": "2023-02-03T22:03:27.960Z"}, {"key": "etablissement_5", "value": "", "created_at": "2023-02-03T22:01:53.360Z", "updated_at": "2023-02-03T22:03:27.960Z"}, {"key": "etablissement_6", "value": "", "created_at": "2023-02-03T22:01:53.360Z", "updated_at": "2023-02-03T22:03:27.960Z"}, {"key": "telechargement_sequence_14

Valeurs uniques : 186695


**Theses data will be completed with interaction_events**  
So, let's drop the column

In [131]:
drop_col('json_metadata')

'Column `json_metadata` dropped. New shape : (198889, 42)'

### Update `anciennete`

In [ ]:
def update_anciennete(row):
    anciennete = row['anciennete']
    created_at = row['created_at']

    if pd.isna(created_at):
        return anciennete

    try:
        if isinstance(created_at, str):
            created_date = pd.to_datetime(created_at)
        else:
            created_date = created_at

        annee_creation = created_date.year
        ecart_annees = 2025 - annee_creation

        if pd.isna(anciennete):
            return ecart_annees

        return anciennete + ecart_annees

    except:
        return anciennete

df_users['anciennete'] = df_users.apply(update_anciennete, axis=1)

### Only need day, not time in `created_at`

In [151]:
df_users['created_at'] = pd.to_datetime(df_users['created_at']).dt.date

## Global view of table

In [133]:
df_users.shape

(198889, 42)

In [134]:
df_users.columns

Index(['id', 'statut_infolettre', 'statut_mailchimp', 'codepostal',
       'anciennete', 'created_at', 'niveau_1ere', 'niveau_2nde', 'niveau_3e',
       'niveau_4e', 'niveau_5e', 'niveau_6e', 'niveau_ash', 'niveau_bac_pro',
       'niveau_cap', 'niveau_ce1', 'niveau_ce2', 'niveau_cm1', 'niveau_cm2',
       'niveau_cp', 'niveau_direction',
       'niveau_formateur_trice_/inspecteur_trice', 'niveau_gs', 'niveau_ms',
       'niveau_post_bac', 'niveau_ps', 'niveau_professeur_e_documentaliste',
       'niveau_segpa', 'niveau_tps', 'niveau_terminale', 'degre', 'maternelle',
       'elementaire', 'college', 'lycee', 'lycee_pro', 'autre',
       'code_postal_etab', 'academie_etab', 'type_etablissement_etab',
       'departement', 'discipline'],
      dtype='object')

In [135]:
df_users.dtypes

id                                            int64
statut_infolettre                             int64
statut_mailchimp                             object
codepostal                                   object
anciennete                                  float64
created_at                                   object
niveau_1ere                                   int64
niveau_2nde                                   int64
niveau_3e                                     int64
niveau_4e                                     int64
niveau_5e                                     int64
niveau_6e                                     int64
niveau_ash                                    int64
niveau_bac_pro                                int64
niveau_cap                                    int64
niveau_ce1                                    int64
niveau_ce2                                    int64
niveau_cm1                                    int64
niveau_cm2                                    int64
niveau_cp   

In [152]:
df_users.sample(10)

,id,statut_infolettre,statut_mailchimp,codepostal,anciennete,created_at,niveau_1ere,niveau_2nde,niveau_3e,niveau_4e,...,elementaire,college,lycee,lycee_pro,autre,code_postal_etab,academie_etab,type_etablissement_etab,departement,discipline
150199,156779,0,subscribed,91150,10.0,2023-09-09,0,0,0,1,...,0,1,0,0,0,91150,Versailles,ECOLE DE NIVEAU ELEMENTAIRE,91,NaN
59457,64162,0,cleaned,93000,4.0,2021-03-10,0,0,0,0,...,1,0,0,0,0,NaN,Créteil,NaN,93,NaN
27777,30806,1,subscribed,NaN,5.0,2020-03-09,0,0,0,0,...,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN
145045,151489,0,subscribed,88204,25.0,2023-08-28,0,0,0,0,...,0,0,1,0,1,88204,Nancy-Metz,LYCEE POLYVALENT,88,Sciences et technologies
8377,9550,1,subscribed,62960,8.0,2018-07-11,0,0,1,1,...,0,1,0,0,0,NaN,Lille,NaN,62,Langues vivantes
74750,79736,1,subscribed,NaN,35.0,2021-08-24,0,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
33783,37473,1,subscribed,14100,31.0,2020-07-28,0,0,0,0,...,1,0,0,0,0,NaN,Normandie,NaN,14,NaN
17481,19306,1,subscribed,NaN,6.0,2019-02-26,0,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
157350,164162,0,subscribed,64000,6.0,2023-10-11,0,0,0,0,...,0,0,0,0,0,64000,Bordeaux,ECOLE DE NIVEAU ELEMENTAIRE,64,NaN
104591,110310,1,unsubscribed,88000,27.0,2022-08-22,0,0,0,0,...,1,0,0,0,0,88000,Nancy-Metz,ECOLE DE NIVEAU ELEMENTAIRE,88,NaN


## Rename columns for understandable names

In [153]:
df_users.columns

Index(['id', 'statut_infolettre', 'statut_mailchimp', 'codepostal',
       'anciennete', 'created_at', 'niveau_1ere', 'niveau_2nde', 'niveau_3e',
       'niveau_4e', 'niveau_5e', 'niveau_6e', 'niveau_ash', 'niveau_bac_pro',
       'niveau_cap', 'niveau_ce1', 'niveau_ce2', 'niveau_cm1', 'niveau_cm2',
       'niveau_cp', 'niveau_direction',
       'niveau_formateur_trice_/inspecteur_trice', 'niveau_gs', 'niveau_ms',
       'niveau_post_bac', 'niveau_ps', 'niveau_professeur_e_documentaliste',
       'niveau_segpa', 'niveau_tps', 'niveau_terminale', 'degre', 'maternelle',
       'elementaire', 'college', 'lycee', 'lycee_pro', 'autre',
       'code_postal_etab', 'academie_etab', 'type_etablissement_etab',
       'departement', 'discipline'],
      dtype='object')

In [ ]:
# df_users = df_users.rename(columns={
#     'codepostal': 'code_postal',
#     'type_etablissement_etab': 'type_etab',
#     'academie_etab': 'academie',
#     'niveau_formateur_trice_/inspecteur_trice': 'niveau_formateur',
#     'niveau_professeur_e_documentaliste': 'niveau_documentaliste'
# })

colonnes_ordre = [
    'id', 'statut_infolettre', 'statut_mailchimp', 'code_postal', 'departement', 'academie',
    'anciennete', 'created_at', 'degre', 'maternelle', 'elementaire', 'college', 'lycee', 'lycee_pro', 'autre',
    'type_etab', 'discipline',
    'niveau_tps',
    'niveau_ps',
    'niveau_ms',
    'niveau_gs',
    'niveau_cp',
    'niveau_ce1',
    'niveau_ce2',
    'niveau_cm1',
    'niveau_cm2',
    'niveau_6e',
    'niveau_5e',
    'niveau_4e',
    'niveau_3e',
    'niveau_2nde',
    'niveau_1ere',
    'niveau_terminale',
    'niveau_cap',
    'niveau_bac_pro',
    'niveau_post_bac',
    'niveau_segpa',
    'niveau_ash',
    'niveau_direction',
    'niveau_formateur',
    'niveau_documentaliste',

]

df_users = df_users[colonnes_ordre]

In [157]:
df_users

,id,statut_infolettre,statut_mailchimp,code_postal,departement,academie,anciennete,created_at,degre,maternelle,...,niveau_1ere,niveau_terminale,niveau_cap,niveau_bac_pro,niveau_post_bac,niveau_segpa,niveau_ash,niveau_direction,niveau_formateur,niveau_documentaliste
0,1,1,subscribed,78770,78,Versailles,24.0,2016-12-31,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,1,unsubscribed,31130,31,Toulouse,9.0,2016-12-31,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,unsubscribed,NaN,NaN,NaN,8.0,2017-01-11,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,subscribed,93260,93,Créteil,12.0,2017-01-13,1,1,...,0,0,0,0,0,0,0,0,0,0
4,5,1,subscribed,75020,75,Paris,8.0,2017-01-13,2,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198884,210716,0,subscribed,69270,69,Lyon,0.0,2025-07-10,1,1,...,0,0,0,0,0,0,0,0,0,0
198885,210717,0,NaN,NaN,NaN,NaN,7.0,2025-07-10,1,0,...,0,0,0,0,0,0,0,0,0,0
198886,210718,1,subscribed,49270,49,Nantes,0.0,2025-07-10,1,1,...,0,0,0,0,0,0,0,0,0,0
198887,210719,0,subscribed,52210,52,Reims,0.0,2025-07-10,1,0,...,0,0,0,0,0,0,0,0,0,0


## Export new users_data

In [158]:
df_users.to_csv("../data/users_cleaned.csv", index=False)

In [159]:
df_users.columns

Index(['id', 'statut_infolettre', 'statut_mailchimp', 'code_postal',
       'departement', 'academie', 'anciennete', 'created_at', 'degre',
       'maternelle', 'elementaire', 'college', 'lycee', 'lycee_pro', 'autre',
       'type_etab', 'discipline', 'niveau_tps', 'niveau_ps', 'niveau_ms',
       'niveau_gs', 'niveau_cp', 'niveau_ce1', 'niveau_ce2', 'niveau_cm1',
       'niveau_cm2', 'niveau_6e', 'niveau_5e', 'niveau_4e', 'niveau_3e',
       'niveau_2nde', 'niveau_1ere', 'niveau_terminale', 'niveau_cap',
       'niveau_bac_pro', 'niveau_post_bac', 'niveau_segpa', 'niveau_ash',
       'niveau_direction', 'niveau_formateur', 'niveau_documentaliste'],
      dtype='object')